# Data



In [1]:
from mini_transformer.data.config import DatasetConfig
from mini_transformer.data.builder import TranslationDatasetBuilder
from mini_transformer.container import MiniTransformerContainer

In [2]:
container = MiniTransformerContainer()
container.init_resources()

## Dataset Builder
From the WMT14 dataset, we will extract training, validation, and test data as follows:
- training: 128 
- validation: 16
- test: 16
We'll oversample by a factor of 5 to ensure that we have a valid number of samples following validation and filtering.

In [3]:
train_config = DatasetConfig(split='train', dataset_size=128, oversample=5)
val_config = DatasetConfig(split='validation', dataset_size=16, oversample=5)
test_config  =  DatasetConfig(split='test', dataset_size=16, oversample=5)

In [4]:
builder_train = TranslationDatasetBuilder(config=train_config)
builder_val = TranslationDatasetBuilder(config=val_config)
builder_test = TranslationDatasetBuilder(config=test_config)


In [ ]:
val = builder_val.build()


Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

[08/21/2025 11:23:00 PM] [INFO] [mini_transformer.data.builder] [log_summary] : 

                             DatasetBuilderObserver                             
                              started_at | 2025-08-21 23:22:56.607089
                                ended_at | 2025-08-21 23:23:00.623940
                                duration | 4.017
                                 dataset | wmt14
                                   split | validation
                                    lang | fr-en
                                    seed | 42
                            dataset_size | 16
                              oversample | 5
                                    seen | 92
                                filtered | 12
                              candidates | 80
                                selected | 16
                          filtered_empty | 0
                          filtered_ratio | 0
                      filtered_src_short | 12
                       filtered_src_lon



                             DatasetBuilderObserver                             
                              started_at | 2025-08-21 23:22:56.607089
                                ended_at | 2025-08-21 23:23:00.623940
                                duration | 4.017
                                 dataset | wmt14
                                   split | validation
                                    lang | fr-en
                                    seed | 42
                            dataset_size | 16
                              oversample | 5
                                    seen | 92
                                filtered | 12
                              candidates | 80
                                selected | 16
                          filtered_empty | 0
                          filtered_ratio | 0
                      filtered_src_short | 12
                       filtered_src_long | 0
                      filtered_tgt_short | 0
                       filter